In [1]:
import matplotlib.pyplot as plt
import h5py
import numpy as np
import pandas as pd
import torch

# Both Jupyter and `pfl` use async. `nest_asyncio` allows `pfl` to run inside the notebook 
import nest_asyncio
nest_asyncio.apply()

# append the root directory to your paths to be able to reach the examples.  
torch.random.manual_seed(1)
np.random.seed(1)

# Always import the `pfl` model first before initializing any `pfl` components to let `pfl` know which Deep Learning framework you will use.
import multiprocessing
# Set multiprocessing start method to "spawn" instead of forkserver (which is the default)
# That is because forkserver does not work on Windows, but spawn does.
def init_multiprocessing():
    try:
        multiprocessing.set_start_method("spawn", force=True)  # Forces "spawn"
    except RuntimeError:
        pass  # Ignore if it's already set

init_multiprocessing()

from pfl.model.pytorch import PyTorchModel

# Define the DP mechanism

In [2]:
# Define Gaussian DP mechanisms for central DP guarantees using three different methods

clipping_bound = 0.5
epsilon = 2
delta = 1e-8
cohort_size = 50
num_epochs = 100 # For DP
central_num_iterations = 10
sampling_probability = 1e-4
is_central = True

http://www.gautamkamath.com/CS860notes/lec5.pdf

Definition 5 on page 3 gives the definition of the parameters for Gaussian distribution where we sample noise for DP.

$\Delta_2^2$ is not included when defining `relative_noise_stddev`, because the clipping bound (which I understand to be $\Delta_2^2$ and also is $\Delta_2^{(f)}$ from definition 3 on page 3) is multiplied on `relative_noise_stddev`, when sampling the noise for DP via `GaussianMechanism.add_noise()`

Source: https://apple.github.io/pfl-research/reference/privacy.html#pfl.privacy.gaussian_mechanism.GaussianMechanism

In [9]:
import math
relative_noise_stddev = 2 * math.log(1.25 / delta) * 1/(epsilon**2)

In [10]:
# define a Gaussian DP mechanism using the PLD privacy accountant
# WARNING: it takes a while for the gaussian_moments_accountant mechanism to be instantiated

from pfl.privacy import (PLDPrivacyAccountant, CentrallyAppliedPrivacyMechanism, GaussianMechanism, LocalPrivacyMechanism)

# define the PLD privacy accountant, which will use the Gaussian noise mechanism
pld_accountant = PLDPrivacyAccountant(
    num_compositions=num_epochs,
    sampling_probability=sampling_probability,
    mechanism='gaussian',
    epsilon=epsilon,
    delta=delta)

# instantiate a Gaussian noise mechanism using the privacy accountant
pld_gaussian_noise_mechanism = GaussianMechanism.from_privacy_accountant(
    accountant=pld_accountant, clipping_bound=clipping_bound)

# wrap the noise mechanism with CentrallyAppliedPrivacyMechanism to make it a central privacy mechanism
pld_central_privacy = CentrallyAppliedPrivacyMechanism(pld_gaussian_noise_mechanism)

## Define a Local DP mechanism

https://apple.github.io/pfl-research/reference/privacy.html#pfl.privacy.gaussian_mechanism.GaussianMechanism

In [11]:
from pfl.metrics import Metrics

class LocallyAppliedPrivacyMechanism(LocalPrivacyMechanism):

    def privatize(self, statistics, name_formatting_fn=..., seed = None):
        # TODO: Implement actual privatization for local DP
        # TODO: Sample some noise (Gaussian: parameters depend on privacy parameters)
        gaussian_mechanism = GaussianMechanism(clipping_bound=clipping_bound, 
                                               relative_noise_stddev=relative_noise_stddev)
        noisy_statistics, metrics = gaussian_mechanism.add_noise(statistics=statistics, 
                                     cohort_size=cohort_size, 
                                     name_formatting_fn=name_formatting_fn)
        # TODO: Add the noise to statistics then send it
        return noisy_statistics, metrics

# Define the model

In [3]:
import torch.nn as nn
import torch.nn.functional as F
from typing import Dict, Optional
from pfl.metrics import Weighted


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


pytorch_model = Net()

loss_fn = torch.nn.CrossEntropyLoss()

def loss(inputs: torch.Tensor, targets: torch.Tensor, eval: bool = False) -> torch.Tensor:
    pytorch_model.eval() if eval else pytorch_model.train()
    return loss_fn(pytorch_model(inputs), targets)

"""
@torch.no_grad()
def metrics(inputs: torch.Tensor,
             targets: torch.Tensor,
             eval: bool = True) -> Dict[str, Weighted]:
    pytorch_model.eval() if eval else pytorch_model.train()
    prediction = pytorch_model(inputs)
    logits = torch.argmax(prediction, dim=1)
    num_samples = len(inputs)
    num_predictions = targets.numel()
    correct = torch.sum(torch.eq((logits > 0.0).float(), targets))

    loss = loss_fn(prediction, targets).item()
    return {
        "loss": Weighted(loss, num_samples),
        "accuracy": Weighted(correct, num_predictions)
    }
"""

@torch.no_grad()
def metrics(inputs: torch.Tensor,
             targets: torch.Tensor,
             eval: bool = True) -> Dict[str, Weighted]:
    pytorch_model.eval() if eval else pytorch_model.train()
    prediction = pytorch_model(inputs)
    logits = torch.argmax(prediction, dim=1)
    num_samples = len(inputs)
    num_predictions = targets.numel()
    correct = torch.sum(torch.eq((logits > 0.0).float(), targets))

    loss = loss_fn(prediction, targets).item()
    return {
        "loss": Weighted(loss, num_samples),
        "accuracy": Weighted(correct, num_samples)
    }


pytorch_model.loss = loss
pytorch_model.metrics = metrics

# Preprocess the data

## Load the data

In [4]:
from pfl.data.dataset import Dataset
import sys

sys.path.append('../flower/flower_normal')
from centralized import load_data
train_set, test_set = load_data()


## Reformat the data

In [5]:
# Split the training dataloader into features and labels
all_features = []
all_labels = []

for features, labels in train_set:
    all_features.append(features)
    all_labels.append(labels)

# Convert to tensors if needed
all_features = torch.cat(all_features, dim=0)
all_labels = torch.cat(all_labels, dim=0)

central_data = Dataset([all_features, all_labels])

In [6]:
print(all_features.shape)
print(all_labels.shape)

torch.Size([50000, 3, 32, 32])
torch.Size([50000])


In [7]:
# Split the validation dataloader into features and labels
val_all_features = []
val_all_labels = []

for features, labels in test_set:
    val_all_features.append(features)
    val_all_labels.append(labels)

# Convert to tensors if needed
val_all_features = torch.cat(val_all_features, dim=0)
val_all_labels = torch.cat(val_all_labels, dim=0)


In [8]:
print(val_all_features.shape)
print(val_all_labels.shape)

torch.Size([10000, 3, 32, 32])
torch.Size([10000])


In [9]:
n_samples = len(train_set.dataset)
print(n_samples)
val_n_samples = len(test_set.dataset)
print(val_n_samples)

50000
10000


# Train the model

## Processing the data

### Creating artificial federated dataset

In [10]:
from pfl.data import ArtificialFederatedDataset, get_data_sampler

# Create data sampler to sample each artificial user dataset as a random subset of the original dataset
data_sampler = get_data_sampler(sample_type="minimize_reuse", max_bound=n_samples)

# Create an artificial federated dataset where each user dataset has constant size such that there are 10 users to distribute among
sample_dataset_len = lambda: int(n_samples/10)
federated_dataset = ArtificialFederatedDataset.from_slices(
    data=[all_features, all_labels], 
    data_sampler=data_sampler,
    sample_dataset_len=sample_dataset_len,
)

In [ ]:
val_data_sampler = get_data_sampler(sample_type="minimize_reuse", max_bound=val_n_samples)

# Create an artificial federated dataset where each user dataset has constant size such that there are of 10 users to distribute among
val_sample_dataset_len = lambda: int(val_n_samples/10)
val_federated_dataset = ArtificialFederatedDataset.from_slices(
    data=[val_all_features, val_all_labels],
    data_sampler=val_data_sampler,
    sample_dataset_len=val_sample_dataset_len
)

### Creating a federated dataset

For the purpose of being able to control the actual number of users, which `ArtificialFederatedDataset` does not allow

<strong>Reflection on designing `FederatedDataset`</strong>

How many datapoints should each user have?
- Every user has 50000/100 = 500 datapoints (easy approach)
- Alternatively, do some randomization on how many datapoints each user has (harder approach)

Should we use the exact same federated dataset in every FL experiment?
- The datapoint will be allocated to the same user in every experiment. This means that user 0 will have the exact same datapoints. User 1 will also have the exact same datapoints and so on... for every single time we run our experiment. This means our federated dataset will be exactly the same every time we run our experiment.

We decided to assume that all users have equally many datapoints and they will each possess the same datapoints every time we run our experiments.

In [10]:
from pfl.data import get_user_sampler, FederatedDataset

I must first decide how many users there are and then the method to how the data is distributed among the users.

In [11]:
# Hyperparameter
n_users = 100
user_dataset_size = int(n_samples / n_users)
print(user_dataset_size)

# Maps user id to user dataset
user_id_to_data = {}

for i in range(n_users):
    start = i*user_dataset_size
    end = start+user_dataset_size
    features = all_features[start:end]
    print(features.shape)
    labels = all_labels[start:end]
    print(labels.shape)
    user_id_to_data[i] = (features, labels)

user_ids = list(user_id_to_data.keys())

user_sampler = get_user_sampler(sample_type="random", user_ids=user_ids)

500
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Size([500, 3, 32, 32])
torch.Size([500])
torch.Siz

In [12]:
federated_dataset = FederatedDataset.from_slices(
    data=user_id_to_data,
    user_sampler=user_sampler)

In [13]:
import importlib
import pfl.data.dataset

importlib.reload(pfl.data.dataset)

from pfl.data.dataset import Dataset

cohorts = federated_dataset.get_cohort(1)
print(f'cohorts: {cohorts}')
user_dataset, _ = next(cohorts)
print(f'user_dataset: {user_dataset}')
len(user_dataset)

cohorts: <generator object FederatedDataset.get_cohort at 0x000001A83E5C7920>
user_dataset: <pfl.data.dataset.Dataset object at 0x000001A81AFD6BC0>


500

In [ ]:
from pfl.internal.ops.selector import get_framework_module as get_ops
from pfl.internal.ops.selector import has_framework_module

def _first_tensor_length(data) -> int:
    if isinstance(data, (list, tuple)):
        return _first_tensor_length(data[0])
    elif isinstance(data, dict):
        return _first_tensor_length(next(iter(data.values())))
    elif has_framework_module() and get_ops().is_tensor(data):
        return get_ops().get_shape(data)[0]
    else:
        assert isinstance(data, np.ndarray), (
            "The data must be of type NumPy array. "
            "If you want to use tensors of a particular framework you "
            "must import its corresponding Model class from "
            "`pfl.model` at the top of your script.")
        return data.shape[0]

_first_tensor_length(all_features[0:10])

True

## Setting up the model

In [14]:
params = [p for p in pytorch_model.parameters() if p.requires_grad]

model = PyTorchModel(pytorch_model, 
                     local_optimizer_create=torch.optim.SGD,
                     central_optimizer=torch.optim.SGD(params, 0.1, momentum=0.9))

In [15]:

from pfl.algorithm import FederatedAveraging, NNAlgorithmParams
from pfl.callback import CentralEvaluationCallback, AggregateMetricsToDisk
from pfl.hyperparam import NNTrainHyperParams, NNEvalHyperParams
from pfl.aggregate.simulate import SimulatedBackend


model_train_params = NNTrainHyperParams(
    local_learning_rate=0.1,
    local_num_epochs=5,
    local_batch_size=32)

# Do full-batch evaluation to run faster.
model_eval_params = NNEvalHyperParams(local_batch_size=None)

evaluation_frequency = 1
algorithm_params = NNAlgorithmParams(
    central_num_iterations=central_num_iterations,
    evaluation_frequency=evaluation_frequency,
    train_cohort_size=cohort_size,
    val_cohort_size=0)

pfl_callbacks = [CentralEvaluationCallback(central_data, model_eval_params, evaluation_frequency), AggregateMetricsToDisk(output_path='pfl_training_metrics/metrics.csv')]

#postprocessors = [LocallyAppliedPrivacyMechanism()]
postprocessors = []

pfl_simulated_backend = SimulatedBackend(
    training_data=federated_dataset,
    val_data=None,
    postprocessors=postprocessors
)


algorithm = FederatedAveraging()

## Run the training

In [ ]:
# PFL training using DP

pfl_model = algorithm.run(
    backend=pfl_simulated_backend,
    model=model,
    algorithm_params=algorithm_params,
    model_train_params=model_train_params,
    model_eval_params=model_eval_params,
    callbacks=pfl_callbacks,
    send_metrics_to_platform=True)

Metrics at iteration 0 ():
    Central val | loss                                : 4.6076517105102536e-05
    Central val | accuracy                            : 0.1
    Central val | number of data points               : 50000
    Train population | number of devices              : 50
    Train population | number of data points          : 500.0
    Train population | loss before local training     : 0.004607966136932373
    Train population | accuracy before local training : 0.10084
    Train population | loss after local training      : 0.00453847583770752
    Train population | accuracy after local training  : 0.10868
    Train population | total weight                   : 1.0
    Number of parameters                              : 62006
    Learning rate                                     : 0.1
Metrics at iteration 1 ():
    Train population | number of devices              : 50
    Train population | number of data points          : 500.0
    Train population | loss before local

KeyboardInterrupt: 

: 